In [1]:
import sys
sys.path.append("..")

from gan import output
sys.modules["output"] = output

from gan.doppelganger import DoppelGANger
from gan.util import add_gen_flag, normalize_per_sample, renormalize_per_sample
from gan.load_data import load_data
from gan.network import DoppelGANgerGenerator, Discriminator, AttrDiscriminator
import os
import tensorflow as tf

C:\Users\mwo2008.54063\.conda\envs\doppelganger\lib\site-packages\tensorflow\python\framework\dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\mwo2008.54063\.conda\envs\doppelganger\lib\site-packages\tensorflow\python\framework\dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\mwo2008.54063\.conda\envs\doppelganger\lib\site-packages\tensorflow\python\framework\dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\mwo2008.54063\.conda\envs\do

In [2]:
sample_len = 1

data_feature, data_attribute, data_gen_flag, data_feature_outputs, data_attribute_outputs = load_data(
    '../../../../../../Master Thesis/data/doppelGANger/final/')
print(data_feature.shape)
print(data_attribute.shape)
print(data_gen_flag.shape)

(191913, 5, 47)
(191913, 38)
(191913, 5)


In [3]:
real_attribute_mask = [True] * len(data_attribute_outputs)

print(real_attribute_mask)
print(data_feature.shape)
print(data_attribute.shape)
print(len(data_attribute_outputs))

[True, True, True, True, True, True, True, True]
(191913, 5, 47)
(191913, 38)
8


In [4]:
data_feature, data_feature_outputs = add_gen_flag(data_feature, data_gen_flag, data_feature_outputs, sample_len)
print(data_feature.shape)
print(len(data_feature_outputs))

(191913, 5, 49)
2


In [5]:
generator = DoppelGANgerGenerator(feed_back=False,
                                  noise=True,
                                  feature_outputs=data_feature_outputs,
                                  attribute_outputs=data_attribute_outputs,
                                  real_attribute_mask=real_attribute_mask,
                                  sample_len=sample_len)
discriminator = Discriminator()

In [6]:
checkpoint_dir = "C:/Users/mwo2008.54063/Documents/BitBucket/synthetic-sequential/generation/NoPrivacy/DoppelGANger/training_IKNL/test/checkpoint"
sample_dir = "C:/Users/mwo2008.54063/Documents/BitBucket/synthetic-sequential/generation/NoPrivacy/DoppelGANger/training_IKNL/test/sample"
time_path = "C:/Users/mwo2008.54063/Documents/BitBucket/synthetic-sequential/generation/NoPrivacy/DoppelGANger/training_IKNL/test/time.txt"

epoch = round((200000*100)/data_feature.shape[0])
batch_size = 100
vis_freq = data_feature.shape[0]
vis_num_sample = 5
d_rounds = 1
g_rounds = 1
d_gp_coe = 10.0
attr_d_gp_coe = 10.0
g_attr_d_coe = 1.0
extra_checkpoint_freq = 5
num_packing = 10

run_config = tf.ConfigProto()

In [7]:
with tf.Session(config=run_config) as sess:
    gan = DoppelGANger(
        sess=sess,
        checkpoint_dir=checkpoint_dir,
        sample_dir=sample_dir,
        time_path=time_path,
        epoch=epoch,
        batch_size=batch_size,
        data_feature=data_feature,
        data_attribute=data_attribute,
        real_attribute_mask=real_attribute_mask,
        data_gen_flag=data_gen_flag,
        sample_len=sample_len,
        data_feature_outputs=data_feature_outputs,
        data_attribute_outputs=data_attribute_outputs,
        vis_freq=vis_freq,
        vis_num_sample=vis_num_sample,
        generator=generator,
        discriminator=discriminator,
        d_gp_coe=d_gp_coe,
        attr_d_gp_coe=attr_d_gp_coe,
        g_attr_d_coe=g_attr_d_coe,
        d_rounds=d_rounds,
        g_rounds=g_rounds,
        num_packing=num_packing,
        extra_checkpoint_freq=extra_checkpoint_freq)
    gan.build()
    gan.load(checkpoint_dir)
    
    total_generate_num_sample = data_feature.shape[0]
    print('samples generated: ', total_generate_num_sample)

    length = int(data_feature.shape[1] / sample_len)
    real_attribute_input_noise = gan.gen_attribute_input_noise(
        total_generate_num_sample)
    addi_attribute_input_noise = gan.gen_attribute_input_noise(
        total_generate_num_sample)
    feature_input_noise = gan.gen_feature_input_noise(
        total_generate_num_sample, length)
    input_data = gan.gen_feature_input_data_free(
        total_generate_num_sample)
    
    features, attributes, gen_flags, lengths = gan.sample_from(
                    real_attribute_input_noise, addi_attribute_input_noise,
                    feature_input_noise, input_data)
    
    print(features.shape)
    print(attributes.shape)

Layers of DoppelGANgerGenerator
[<tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/linear/dense/kernel:0' shape=(5, 100) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/linear/dense/bias:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/batch_norm/beta:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/batch_norm/gamma:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/batch_norm/moving_mean:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/batch_norm/moving_variance:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer1/linear/dense/kernel:0' shape=(100, 100) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer1/linear/dense/bias:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_real

In [8]:
import numpy as np

In [9]:
# Save
np.savez('./generated_data/generated_data.npz',
         data_feature=features,
         data_attribute=attributes,
         data_gen_flag=gen_flags)